# Create a point defect pair in a bulk material

Create a pair of point defects: substitution, vacancy, or interstitial in a bulk material.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook. 
1. Set notebook parameters in cell 1.1. below (or use the default values) -- to create multiple defects, fill `DEFECT_CONFIGS` with a list of dictionaries. 
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 

## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defects parameters 
Defect Configuration parameters are described in [Defect Configuration](https://github.com/Exabyte-io/made/blob/8196b759242551c77d1791bf5bd2f4150763cfef/src/py/mat3ra/made/tools/build/defect/configuration.py#L102).

In [ ]:
from types import SimpleNamespace

# Selected material will be used as a unit cell to create a supercell first.
SUPERCELL_MATRIX = [[3, 0, 0], [0, 3, 0], [0, 0, 3]]

# List of dictionaries with defect parameters
PRIMARY_DEFECT_CONFIG = SimpleNamespace(
    defect_type="vacancy",
    coordinate=[0.5, 0.5, 0.5],  # Approx. coord that will be resolved to the closest site
    use_cartesian_coordinates=False,  # Use cartesian or crystal coordinates
    # "site_id": 0, # Index of the atom in the host material
    # "coordinate": None, # Exact position (override the approximate coordinate)
)

SECONDARY_DEFECT_CONFIG = SimpleNamespace(
    defect_type="substitution",
    approximate_coordinate=[0.15, 0.15, 0.15],  # Approx. coord that will be resolved to the closest site
    chemical_element="C",  # Element to substitute
    # "site_id": 0, # Index of the atom in the host material
    # "coordinate": None, # Exact position (override the approximate coordinate)
)

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("")

### 1.3. Get input materials
Materials are loaded with `get_materials()`.

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

### 1.4. Create and preview Supercell

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.helpers import create_supercell

unit_cell = materials[0]
supercell = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create the Defect
### 2.1. Initialize Configuration and Builder parameters

In [ ]:
from mat3ra.made.tools.helpers import create_defect_pair

material_with_defect = create_defect_pair(
    material=supercell,
    defect_type_1=PRIMARY_DEFECT_CONFIG.defect_type,
    coordinate_1=PRIMARY_DEFECT_CONFIG.coordinate,
    element_1=PRIMARY_DEFECT_CONFIG.chemical_element if hasattr(PRIMARY_DEFECT_CONFIG, "chemical_element") else None,
    placement_method_1=PRIMARY_DEFECT_CONFIG.placement_method if hasattr(PRIMARY_DEFECT_CONFIG,
                                                                         'placement_method') else None,
    defect_type_2=SECONDARY_DEFECT_CONFIG.defect_type,
    coordinate_2=SECONDARY_DEFECT_CONFIG.approximate_coordinate,
    element_2=SECONDARY_DEFECT_CONFIG.chemical_element if hasattr(SECONDARY_DEFECT_CONFIG,
                                                                  "chemical_element") else None,
    placement_method_2=SECONDARY_DEFECT_CONFIG.placement_method if hasattr(SECONDARY_DEFECT_CONFIG,
                                                                           'placement_method') else None,
)

## 3. Visualize Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}],
          rotation="-90x")
visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials([material_with_defect])